In [30]:
    import pandas as pd

    train = pd.read_csv(r"F:\Machine_learning\HackerEarth\train.csv")
    test = pd.read_csv(r"F:\Machine_learning\HackerEarth\test.csv")

    train.columns
    train.dtypes
    # ALL features are either float or integers. and the output is categorical into 4 values. which will be mapped to 1,2,3,4
    train.Violations.value_counts() # this is categorical Variable
    train.Accident_Type_Code.value_counts() # This is a Categorical variable


    train = pd.get_dummies(train, columns =['Accident_Type_Code','Violations'])
    test = pd.get_dummies(test, columns =['Accident_Type_Code','Violations'])

    train.columns
    train = train[['Accident_ID', 'Safety_Score', 'Days_Since_Inspection',
           'Total_Safety_Complaints', 'Control_Metric', 'Turbulence_In_gforces',
           'Cabin_Temperature', 'Max_Elevation', 'Adverse_Weather_Metric',
           'Accident_Type_Code_1', 'Accident_Type_Code_2',
           'Accident_Type_Code_3', 'Accident_Type_Code_4', 'Accident_Type_Code_5',
           'Accident_Type_Code_6', 'Accident_Type_Code_7', 'Violations_0',
           'Violations_1', 'Violations_2', 'Violations_3', 'Violations_4',
           'Violations_5','Severity']]
    # 'Severity' is sent to last columns as that is the output. Accident_ID is made first columns since thats the output format.

    train.Severity.value_counts().sum() # 10000 values
    train.shape # 10000 x 23
    # to confirm the values in the output and shape of the train data.
    train.Severity.value_counts() # Distribution of the data along output types.

    # Mapping all 4 types to numeric values. 
    train.Severity = train.Severity.map({'Highly_Fatal_And_Damaging':0,
                                         'Significant_Damage_And_Serious_Injuries':1,
                                         'Minor_Damage_And_Injuries':2,
                                         'Significant_Damage_And_Fatalities':3},)
    test.columns
    # Adding Accident_ID as the first column to the test data.
    test = test[['Accident_ID','Safety_Score', 'Days_Since_Inspection', 'Total_Safety_Complaints',
           'Control_Metric', 'Turbulence_In_gforces', 'Cabin_Temperature',
           'Max_Elevation', 'Adverse_Weather_Metric',
           'Accident_Type_Code_1', 'Accident_Type_Code_2', 'Accident_Type_Code_3',
           'Accident_Type_Code_4', 'Accident_Type_Code_5', 'Accident_Type_Code_6',
           'Accident_Type_Code_7', 'Violations_0', 'Violations_1', 'Violations_2',
           'Violations_3', 'Violations_4', 'Violations_5']]

    train.shape, test.shape # one extra column in train data which is the output.
    # since ID's are not the part of the data input, they are saved into different columns and later will mapped onto
    # the output data for submission.
    Acc_ID_Train = train.Accident_ID
    Acc_ID_Test = test.Accident_ID
    # Dropping the Accident ID
    train = train.iloc[:,1:]
    test = test.iloc[:,1:]


    # Create X_train, y_train and we have test data withour output.
    X = train.iloc[:,0:-1]
    y = train.iloc[:,[-1]] # severityb

    # X.shape, test.shape


In [31]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 0)


# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Fitting XGBoost to the Training set
from xgboost import XGBClassifier
classifier = XGBClassifier(objective='multi:softmax')
# objective='multi:softmax' for multiclass classifier.
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

C:\Users\spens\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\spens\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [32]:
cm

array([[262,   7,  32,   8],
       [ 14, 244,  36,   5],
       [ 11,  13, 202,  17],
       [ 10,   9,  14, 116]], dtype=int64)

In [33]:
from sklearn import datasets, linear_model, metrics 
metrics.accuracy_score(y_test, y_pred)*100

82.399999999999991

In [34]:
test = sc.transform(test)
y_pred_sub = classifier.predict(test)

In [35]:
y_pred_sub

array([0, 3, 1, ..., 1, 1, 0], dtype=int64)

In [36]:
sub = pd.DataFrame(Acc_ID_Test)
sub['Severity'] = y_pred_sub.tolist()
sub.Severity = sub.Severity.map({0:'Highly_Fatal_And_Damaging',
                   1:'Significant_Damage_And_Serious_Injuries',
                   2:'Minor_Damage_And_Injuries',
                   3:'Significant_Damage_And_Fatalities'})
sub.to_csv(r"F:\Machine_learning\HackerEarth\sample_submission_xgb.csv",index = None, header=True)